<a href="https://colab.research.google.com/github/FabioBoccia/Progetto_ESM/blob/generation-hyperparameters-tuning/Jupyter%20Notebooks/generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taming Transformers

This notebook is a minimal working example to generate faces images as in [Taming Transformers for High-Resolution Image Synthesis](https://github.com/CompVis/taming-transformers).

## Setup
The setup code in this section was written to be [run in a Colab environment](https://colab.research.google.com/github/CompVis/taming-transformers/blob/master/scripts/taming-transformers.ipynb).
It should be run only once per runtime.
We first clone the repository and upload the model checkpoint and config from Google Drive

In [1]:
%cd /content
%rm -r sample_data
!git clone https://github.com/CompVis/taming-transformers
%cd taming-transformers
!mkdir -p logs

from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/2021-04-23T18-19-01_ffhq_transformer logs/

/content
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1335, done.
remote: Total 1335 (delta 0), reused 0 (delta 0), pack-reused 1335
Receiving objects: 100% (1335/1335), 409.77 MiB | 40.82 MiB/s, done.
Resolving deltas: 100% (278/278), done.
/content/taming-transformers
Mounted at /content/drive


Next, we install required dependencies.

In [1]:
!pip install --upgrade omegaconf einops transformers pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Loading the model

We load and print the config.

In [2]:
import sys
%cd /content/taming-transformers
sys.path.append(".")
from omegaconf import OmegaConf
config_path = "/content/taming-transformers/logs/2021-04-23T18-19-01_ffhq_transformer/configs/2021-04-23T18-19-01-project.yaml"
config = OmegaConf.load(config_path)
import yaml
print(yaml.dump(OmegaConf.to_container(config)))

/content/taming-transformers
data:
  params:
    batch_size: 24
    num_workers: 24
    train:
      params:
        size: 256
      target: taming.data.faceshq.FFHQTrain
    validation:
      params:
        size: 256
      target: taming.data.faceshq.FFHQValidation
  target: cutlit.DataModuleFromConfig
model:
  base_learning_rate: 0.0625
  params:
    cond_stage_config: __is_unconditional__
    first_stage_config:
      params:
        ddconfig:
          attn_resolutions:
          - 16
          ch: 128
          ch_mult:
          - 1
          - 1
          - 2
          - 2
          - 4
          double_z: false
          dropout: 0.0
          in_channels: 3
          num_res_blocks: 2
          out_ch: 3
          resolution: 256
          z_channels: 256
        embed_dim: 256
        lossconfig:
          target: taming.modules.losses.vqperceptual.DummyLoss
        n_embed: 1024
      target: taming.models.vqgan.VQModel
    first_stage_key: image
    transformer_config:
   

Instantiate the model and load the checkpoint.

In [3]:
from taming.models.cond_transformer import Net2NetTransformer
import torch
model = Net2NetTransformer(**config.model.params)
ckpt_path = "/content/taming-transformers/logs/2021-04-23T18-19-01_ffhq_transformer/checkpoints/last.ckpt"
sd = torch.load(ckpt_path, map_location="cuda")["state_dict"]
missing, unexpected = model.load_state_dict(sd, strict=False)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Using no cond stage. Assuming the training is intended to be unconditional. Prepending 0 as a sos token.


In [4]:
model.cuda().eval()
torch.set_grad_enabled(False)

## Generate image samples
Generated images will be saved in `logdir/samples/top_k_{top_k}_top_p_{top_p}/`
as PNGs

if you wish to display the generated images instead of saving them in logdir, 
set save = False


In [ ]:
num_samples = 1000
batch_size = 3
top_k = 300
top_p = 1.0
temperature = 1.0

save = True
logdir="/content/drive/MyDrive/"

In [ ]:
from scripts.sample_fast import sample_unconditional
from tqdm import tqdm
from PIL import Image
import numpy as np

def show_image(s):
  s = s.detach().cpu().numpy().transpose(0,2,3,1)[0]
  s = ((s+1.0)*127.5).clip(0,255).astype(np.uint8)
  s = Image.fromarray(s)
  display(s)

@torch.no_grad()
def run_live(model, batch_size, temperature, top_k, unconditional=True, num_samples=50000, top_p=None):
    print(f"Running in unconditional sampling mode, producing {num_samples} samples.")
    batches = [batch_size for _ in range(num_samples//batch_size)] + [num_samples % batch_size]
    for n, bs in tqdm(enumerate(batches), desc="Sampling"):
        if bs == 0: break
        logs = sample_unconditional(model, batch_size=bs, temperature=temperature, top_k=top_k, top_p=top_p)
        show_image(logs["samples"])

In [ ]:
from scripts.sample_fast import run
import os 

if save == True:
    logdir = os.path.join(logdir, "samples", f"top_k_{top_k}_top_p_{top_p}")
    print(f"Logging to {logdir}")
    os.makedirs(logdir, exist_ok=True)
    run(logdir, model, batch_size, temperature, top_k, unconditional=model.be_unconditional, num_samples=num_samples, top_p=top_p)
else:
    print(f"Generating with top_k = {top_k}, top_p = {top_p}")
    run_live(model, batch_size, temperature, top_k, unconditional=model.be_unconditional, num_samples=num_samples, top_p=top_p)

Logging to /content/drive/MyDrive/samples/top_k_300_top_p_1.0
Running in unconditional sampling mode, producing 3000 samples.


Sampling: 196it [41:39, 12.72s/it]

## Tuning Hyperparameters


In [48]:
%cd /content
!wget --no-check-certificate --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/ffhq_real.zip
!unzip -q ffhq_real.zip
#!rm -d 0_real
#!rm ffhq_real.zip
%mkdir -p /content/ffhq/real
%mkdir -p /content/ffhq/synthesized

In [97]:
from scripts.sample_fast import run
from tqdm import tqdm
from PIL import Image
import numpy as np
import os 
import tensorflow as tf

In [141]:
num_samples = 3
## 
batch_size = 3

trainmodel = tf.keras.models.load_model('/content/drive/MyDrive/my_keras_model.h5')

logdir="/content/ffhq/synthesized/"
top_k = tf.Variable(300, name = 'top_k', dtype = tf.float32)
top_p = tf.Variable(1.0, name = 'top_p', dtype = tf.float32)
temperature = tf.Variable(1.0, name = 'temperature', dtype = tf.float32)

def loss():
    %rm /content/ffhq/synthesized/*
    %cd /content/ffhq/real/
    !mv * /content/0_real/
    %cd /content/0_real/
    !mv $(ls | shuf -n 3) ../ffhq/real/
    temp = temperature.numpy()
    k = int(top_k.numpy())
    p = top_p.numpy()
    run(logdir, model, batch_size, temp, k, unconditional=model.be_unconditional, num_samples=num_samples, top_p=p)
    dataset = tf.keras.utils.image_dataset_from_directory(
        '/content/ffhq/',
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=5,
        image_size=(256, 256),
        shuffle=True,
        validation_split=0,
        interpolation='bilinear',
        crop_to_aspect_ratio=True,
    )
    loss, AUC, accuracy = trainmodel.evaluate(dataset)
    return AUC

<tf.Variable 'top_p:0' shape=() dtype=float32, numpy=1.0>


In [147]:
# maximize false negatives (predicted real)
# https://www.tutorialspoint.com/tensorflow/tensorflow_gradient_descent_optimization.htm


optimizer = tf.keras.optimizers.SGD()
vars = [top_k, top_p, temperature]
with tf.GradientTape() as tape:
    tape.watch([AUC, vars])
    AUC = loss()
    grad = tape.gradient(AUC, vars)
train = optimizer.apply_gradients(zip(grad, vars))
#train = optimizer.minimize(loss, [top_k, top_p, temperature])

# def optimize():
#    with tf.Session() as session:
#       session.run(init)
#       print("starting at", "- top_k:", session.run(top_k), "- top_p:", session.run(top_p), "- temperature:", session.run(top_k), "- false_negatives:", session.run(to_maximize))
#       
#       for step in range(10):
#          session.run(train)
#          print("step", step, "- top_k:", session.run(top_k), "- top_p:", session.run(top_p), "- temperature:", session.run(top_k), "- false_negatives:", session.run(to_maximize))

/content/ffhq/real
/content/0_real
Running in unconditional sampling mode, producing 3 samples.


Sampling: 1it [00:07,  7.81s/it]


Found 6 files belonging to 2 classes.
<tf.Variable 'top_p:0' shape=() dtype=float32, numpy=1.0>
2/2 [==============================] - 0s 20ms/step - loss: 0.2673 - auc: 0.8889 - accuracy: 0.6667
[<tf.Variable 'top_k:0' shape=() dtype=float32, numpy=300.0>, <tf.Variable 'top_p:0' shape=() dtype=float32, numpy=1.0>, <tf.Variable 'temperature:0' shape=() dtype=float32, numpy=1.0>]


ValueError: ignored

In [ ]:
optimize()